In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('data_scientist_salary.csv')

In [3]:
# company name
df['company_txt'] = df['company'].apply(lambda x: x.split('\n')[0])

In [4]:
# rating
df['without_rating'] = df['company'].apply(lambda x: 1 if len(x.split('\n'))==1 else 0)
df['rating'] = df.apply(lambda x: -1 if x['without_rating']==1 else x['company'].split('\n')[1], axis = 1)

In [5]:
# state
df['without_city'] = df['location'].apply(lambda x: 1 if len(x.split(','))==1 else 0)
df['state'] = df.apply(lambda x: x['location'] if x['without_city']==1 else x['location'].split(',')[1], axis = 1)

In [6]:
# parse job description

# python
# hadoop
# spark
# tableau
# tensorflow
# aws
# excel
df['python'] = df['job description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['hadoop'] = df['job description'].apply(lambda x: 1 if 'hadoop' in x.lower() else 0)
df['spark'] = df['job description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df['tableau'] = df['job description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
df['tensorflow'] = df['job description'].apply(lambda x: 1 if 'tensorflow' in x.lower() else 0)
df['aws'] = df['job description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df['excel'] = df['job description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

In [7]:
# salary
df = df.dropna(subset=['salary estimate']) # delete rows without salary data

# get rid of per hour
df['hourly'] = df['salary estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
minus_hour = df['salary estimate'].apply(lambda x: x.lower().replace('per hour', ''))

# T stands for thousand and L stands for 10 thousands
minus_TLD = minus_hour.apply(lambda x: x.lower().replace('$', '').replace('t', '').replace('l', '00'))
minus_TLD = minus_TLD.apply(lambda x: x.split('(')[0])

# get min & max salary and average them
df['min_salary'] = minus_TLD.apply(lambda x: float(x.split('-')[0]))
df['max_salary'] = minus_TLD.apply(lambda x: float(x.split('-')[1]) if len(x.split('-')) == 2 else float(x.split('-')[0]))
df['avg_salary'] = (df['min_salary']+df['max_salary'])/2

# multiply per hour salary by 2 to get an estimate of annual salary
df['avg_salary'] = df.apply(lambda x: x['avg_salary']*2 if x['hourly']==1 else x['avg_salary'], axis = 1)

In [8]:
# company found
df['age'] = 2023 - df['company_founded']

In [9]:
# parse job title
title_rank = {'senior': ['senior', 'sr', 'lead', 'principal', 'manager'],
              'director': ['director','president'],
             'junior': ['junior', 'jr']}

title_dict = {'machine learning engineer': ['machine learning', 'deep learning', 'nlp'],
             'data analyst': ['data analyst', 'data analytics', 'data analysis'],
             'data scientist': ['data scientist','data science'],
             'data engineer': ['data engineer','data architect']}

def simplify_title(title, title_dict):
    for key in title_dict:
        for t in title_dict[key]:
            if t in title.lower():    
                return key
    return 'na'

def get_rank(title, title_rank):
    for key in title_rank:
        for r in title_rank[key]:
            if r in title.lower():    
                return key
    return 'na'

df['simp_title'] = df['job title'].apply(lambda x: simplify_title(x, title_dict))
df['rank'] = df['job title'].apply(lambda x: get_rank(x, title_rank))

In [10]:
df.drop(['Unnamed: 0'], inplace=True, axis = 1)
df.head()

,company,job title,location,job description,salary estimate,company_size,company_type,company_sector,company_industry,company_founded,...,tensorflow,aws,excel,hourly,min_salary,max_salary,avg_salary,age,simp_title,rank
0,Oasis Systems LLC\n4.4,Senior Python Data Scientist,"Washington, DC",Overview:\nOasis Systems has an exciting oppor...,$99T - $1L (Glassdoor Est.),1001 to 5000 Employees,Company - Private,Aerospace & Defence,Aerospace & Defence,1997.0,...,0,0,1,0,99.0,100.0,99.5,26.0,data scientist,senior
1,American Family Insurance\n3.2,"Principal Data Scientist - Personal Lines, Aut...","Madison, WI","At American Family Insurance, we believe peopl...",$3L (Employer Est.),5001 to 10000 Employees,Company - Private,Insurance,Insurance Carriers,1927.0,...,0,1,0,0,300.0,300.0,300.0,96.0,data scientist,senior
2,Scottish Rite for Children\n3.9,Senior Healthcare Data Analyst,"Dallas, TX",Our patients are our number one priority! We'r...,$63T - $92T (Glassdoor Est.),1001 to 5000 Employees,Non-profit Organisation,Healthcare,Healthcare Services & Hospitals,1921.0,...,0,0,0,0,63.0,92.0,77.5,102.0,data analyst,senior
3,Northrop Grumman\n4.0,Principal Data Scientist,"Redondo Beach, CA",Requisition ID: R10086374\nCategory: Research ...,$1L - $2L (Employer Est.),10000+ Employees,Company - Public,Aerospace & Defence,Aerospace & Defence,1939.0,...,1,1,0,0,100.0,200.0,150.0,84.0,data scientist,senior
4,Cirkul Inc\n3.4,Lead Data Scientist,"Watertown, MA","About Cirkul, Inc.\nCirkul is a venture backed...",$1L - $2L (Glassdoor Est.),201 to 500 Employees,Company - Private,Manufacturing,Food & Beverage Manufacturing,2016.0,...,0,0,1,0,100.0,200.0,150.0,7.0,data scientist,senior


In [11]:
df.to_csv('cleaned_ds_job.csv', index = None)